In [1]:
from utils_processing.utils import *
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import combinations

In [2]:
walking_files = ['Activity-Data/Walking/01/Walking01Pressure.csv', 'Activity-Data/Walking/2/Walking2Pressure.csv', 
                 'Activity-Data/Walking/3/Walking3Pressure.csv', 'Activity-Data/Walking/04/Walking04Pressure.csv',
                 'Activity-Data/Walking/5/Walking5Pressure.csv']

In [3]:
climbing_files = ['Activity-Data/Climbing_Stairs/1/Climbing_Stairs1Pressure.csv', 'Activity-Data/Climbing_Stairs/2/Climbing_Stairs2Pressure.csv', 
                 'Activity-Data/Climbing_Stairs/3/Climbing_Stairs3Pressure.csv', 'Activity-Data/Climbing_Stairs/5/Climbing_Stairs5Pressure.csv',
                 'Activity-Data/Climbing_Stairs/6/Climbing_Stairs6Pressure.csv', 'Activity-Data/Climbing_Stairs/7/Climbing_Stairs7Pressure.csv']

In [5]:
downstairs_files = ['Activity-Data/Downstairs/1/Downstairs1Pressure.csv', 'Activity-Data/Downstairs/2/Downstairs2Pressure.csv', 
                 'Activity-Data/Downstairs/3/Downstairs3Pressure.csv', 'Activity-Data/Downstairs/4/Downstairs4Pressure.csv',
                 'Activity-Data/Downstairs/5/Downstairs5Pressure.csv', 'Activity-Data/Downstairs/6/Downstairs6Pressure.csv']

In [7]:
walking_frame = read_concatenate_files(walking_files, usecols=[0,1,2])
walking_frame.head()

,timestamp,pressure,UUID
0,1503925783809,963.727173,1b63307ec4900338
1,1503925785588,963.747253,1b63307ec4900338
2,1503925786383,963.760925,1b63307ec4900338
3,1503925786681,963.762085,1b63307ec4900338
4,1503925786897,963.759583,1b63307ec4900338


In [8]:
climbing_frame = read_concatenate_files(climbing_files + downstairs_files, usecols=[0,1,2])
climbing_frame.head()

,timestamp,pressure,UUID
0,1503926233330,963.686096,1b63307ec4900338
1,1503926233527,963.686096,1b63307ec4900338
2,1503926233726,963.682556,1b63307ec4900338
3,1503926233921,963.684448,1b63307ec4900338
4,1503926234223,963.679504,1b63307ec4900338


In [10]:
w_windows, w_ts_windows = create_sliding_windows_by_time(walking_frame, 1, 3000)
c_windows, c_ts_windows = create_sliding_windows_by_time(climbing_frame, 1, 3000)
print(len(w_windows), len(w_ts_windows))
print(len(c_windows), len(c_ts_windows))

59 59
43 43


In [13]:
print(len(climbing_frame), len(walking_frame))

633 884


In [11]:
print([i for i in c_ts_windows])

[[1503926233330, 1503926233527, 1503926233726, 1503926233921, 1503926234223, 1503926234519, 1503926234818, 1503926235028, 1503926235271, 1503926235409, 1503926235809, 1503926236005, 1503926236204], [1503926236421, 1503926236498, 1503926236700, 1503926236798, 1503926237027, 1503926237094, 1503926237310, 1503926237391, 1503926237589, 1503926237690, 1503926237917, 1503926238090, 1503926238212, 1503926238281, 1503926238479, 1503926238681, 1503926238878, 1503926239073, 1503926239278], [1503926239473, 1503926239577, 1503926239681, 1503926239768, 1503926240068, 1503926240194, 1503926240364, 1503926240466, 1503926240662, 1503926241002, 1503926241258, 1503926241463, 1503926241650, 1503926241947, 1503926242272, 1503926242450], [1503926242741, 1503926243081, 1503926243135, 1503926243338, 1503926243533, 1503926243734, 1503926243931, 1503926244158, 1503926244229, 1503926244422, 1503926244620, 1503926244849, 1503926245116, 1503926245327, 1503926245613], [1503926247788, 1503926247892, 1503926248084, 

In [12]:
walking_frame['label'] = 0
climbing_frame['label'] = 1
print(climbing_frame.tail())
print(walking_frame.tail())

        timestamp    pressure             UUID   label
49  1503926812832  963.583496  1b63307ec4900338      1
50  1503926813127  963.592468  1b63307ec4900338      1
51  1503926813327  963.607849  1b63307ec4900338      1
52  1503926813679  963.613464  1b63307ec4900338      1
53  1503926813935  963.625732  1b63307ec4900338      1
         timestamp    pressure             UUID   label
159  1503926687534  962.399292  1b63307ec4900338      0
160  1503926687934  962.394958  1b63307ec4900338      0
161  1503926688153  962.395081  1b63307ec4900338      0
162  1503926688525  962.405029  1b63307ec4900338      0
163  1503926688826  962.408813  1b63307ec4900338      0


In [14]:
w_iqr = compute_inter_quartile_distance_window(w_windows)
c_iqr = compute_inter_quartile_distance_window(c_windows)

In [15]:
w_std = compute_std_deviation_window(w_windows)
c_std = compute_std_deviation_window(c_windows)

In [16]:
w_derivative = compute_sum_derivative_window(w_windows, w_ts_windows)
c_derivative = compute_sum_derivative_window(c_windows, c_ts_windows)

In [17]:
print(len(w_iqr),len(c_iqr), len(w_std), len(c_std), len(w_derivative), len(c_derivative))

58 43 58 43 58 43


In [19]:
print(type(w_derivative[0]), type(w_std[1]), type(w_iqr[2]))

<class 'numpy.float64'> <class 'numpy.float64'> <class 'numpy.float64'>


In [20]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder

In [29]:
w_feature_vector = np.column_stack((np.array(w_std).reshape(-1, 1), np.array(w_derivative).reshape(-1, 1), 
                                   np.array(w_iqr).reshape(-1, 1)))
print(type(w_feature_vector), len(w_feature_vector), w_feature_vector.shape)

<class 'numpy.ndarray'> 58 (58, 3)


In [30]:
c_feature_vector = np.column_stack((np.array(c_std).reshape(-1, 1), np.array(c_derivative).reshape(-1, 1), 
                                   np.array(c_iqr).reshape(-1, 1)))
print(type(c_feature_vector), len(c_feature_vector), c_feature_vector.shape)

<class 'numpy.ndarray'> 43 (43, 3)


In [33]:
labels_vector = np.stack((np.array([0 for i in range(len(w_feature_vector))]).reshape(-1,1), 
                          np.array([1 for i in range(len(c_feature_vector))]).reshape(-1,1)))
print(type(labels_vector), len(labels_vector), labels_vector.shape)

ValueError: all input arrays must have the same shape

In [37]:
c_labels = np.array([1 for i in range(len(c_feature_vector))]).reshape(-1,1)
print(c_labels.shape)
w_labels = np.array([0 for i in range(len(w_feature_vector))]).reshape(-1,1)
print(w_labels.shape)
labels_vector = np.stack((w_labels,c_labels))
print(type(labels_vector), len(labels_vector), labels_vector.shape)

(43, 1)
(58, 1)


ValueError: all input arrays must have the same shape

In [40]:
print(type(w_labels[5]))

<class 'numpy.ndarray'>


In [47]:
c_labels = np.full((len(c_feature_vector),1), 1)
w_labels = np.full((len(w_feature_vector),1), 0)
print(c_labels.shape, w_labels.shape)

(43, 1) (58, 1)


In [51]:
labels_vector = np.concatenate((w_labels, c_labels))
print(type(labels_vector), len(labels_vector), labels_vector.shape)

<class 'numpy.ndarray'> 101 (101, 1)


In [54]:
feature_vectors = np.concatenate((w_feature_vector, c_feature_vector))
print(type(feature_vectors), len(feature_vectors), feature_vectors.shape)

<class 'numpy.ndarray'> 101 (101, 3)


In [45]:
a = np.zeros(10)
b = np.ones(10)
print(a.shape, b.shape)
print(a, b)
a = a.reshape(-1, 1)
b = b.reshape(1,-1)
print(a, b)


(10,) (10,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] [[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]


In [65]:
X = np.copy(feature_vectors)
Y = np.copy(labels_vector)
Y = Y.ravel()

In [57]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [76]:
print(Y.shape, Y)

(101,) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
ohe = OneHotEncoder()
Yt = Y.reshape(-1,1)
print(Yt)
Ytt = ohe.fit_transform(Yt)
print(Ytt.shape)
print(Ytt)

In [ ]:
print(Y,Y.reshape(-1,1))

In [81]:
from sklearn.metrics import confusion_matrix
kf = KFold(n_splits=5)
# print(list(kf.split(X)))
# print(list(kf.split(Y)))
for train, test in kf.split(X):
    XTrain, XTest, YTrain, YTest = X[train], X[test], Y[train], Y[test]
    clf = RandomForestClassifier(n_estimators=500, random_state=seed)
    clf = clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
    print(confusion_matrix(YTest, YPred))
    score = diff[diff == 0].size
    print((100.0 * score)/(YPred.size))

[[19  2]
 [ 0  0]]
90.47619047619048
[[19  1]
 [ 0  0]]
95.0
[[14  3]
 [ 1  2]]
80.0
[[ 0  0]
 [ 5 15]]
75.0
[[ 0  0]
 [ 6 14]]
70.0


In [83]:
skf = StratifiedKFold(n_splits=5)
# print(list(kf.split(X)))
# print(list(kf.split(Y)))
for train, test in skf.split(X,Y):
    XTrain, XTest, YTrain, YTest = X[train], X[test], Y[train], Y[test]
    clf = RandomForestClassifier(n_estimators=1500, random_state=seed)
    clf = clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
#     print(diff)
    print(confusion_matrix(YTest, YPred))
    score = diff[diff == 0].size
    print((100.0 * score)/(YPred.size))

[[12  0]
 [ 2  7]]
90.47619047619048
[[10  2]
 [ 1  8]]
85.71428571428571
[[12  0]
 [ 3  6]]
85.71428571428571
[[10  1]
 [ 0  8]]
94.73684210526316
[[10  1]
 [ 1  7]]
89.47368421052632
